In [4]:
# Generate the synthetic data
x1 = np.sin(np.arange(0, 9, 0.01))
x2 = np.cos(np.arange(0, 9, 0.01))
x3 = x1**2
x4 = (x1+x2)/2
tmp = np.ones((900,))

Xtrain_c = x3[0:5]
Xtrain_r = tmp[0:5]
Ytrain_c = x4[0:1]
Ytrain_r = tmp[0:1]

for i in range(1,100):
    Xtrain_c = np.vstack((Xtrain_c, x3[i:i+5]))
    Xtrain_r = np.vstack((Xtrain_r, tmp[i:i+5]))
    Ytrain_c = np.vstack((Ytrain_c, x4[i:i+1]))
    Ytrain_r = np.vstack((Ytrain_r, tmp[i:i+1]))

# Parameters
input_dim = 5
output_dim = 1
num_hidden_layers = 2
num_units = [200, 200]
act = ['relu', 'relu']
beta = 0.5

# Get model
model = get_model(input_dim, output_dim, num_units, act, beta, num_hidden_layers)

# Train
model.fit(x=[Xtrain_c, Xtrain_r], y=[Ytrain_c, Ytrain_r], epochs=10)

Epoch 1/10
100/100 [==============================] - 1s 10ms/step - loss: 3.5982 - dense_17_loss: 2.6060 - dense_18_loss: 0.0421 - dense_17_accuracy: 0.0000e+00 - dense_18_accuracy: 1.0000
Epoch 2/10
100/100 [==============================] - 0s 160us/step - loss: 2.1682 - dense_17_loss: 1.2155 - dense_18_loss: 0.0219 - dense_17_accuracy: 0.0000e+00 - dense_18_accuracy: 1.0000
Epoch 3/10
100/100 [==============================] - 0s 162us/step - loss: 1.0881 - dense_17_loss: 0.2661 - dense_18_loss: 0.0198 - dense_17_accuracy: 0.0000e+00 - dense_18_accuracy: 1.0000
Epoch 4/10
100/100 [==============================] - 0s 164us/step - loss: 1.0417 - dense_17_loss: 0.2098 - dense_18_loss: 0.0226 - dense_17_accuracy: 0.0000e+00 - dense_18_accuracy: 1.0000
Epoch 5/10
100/100 [==============================] - 0s 160us/step - loss: 1.2980 - dense_17_loss: 0.6110 - dense_18_loss: 0.0042 - dense_17_accuracy: 0.0000e+00 - dense_18_accuracy: 1.0000
Epoch 6/10
100/100 [==========================

In [1]:
from imlp import iAct, iLoss, get_model
import numpy as np
import pandas as pd
import os
from tqdm import trange
import json

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

Using TensorFlow backend.


In [2]:
method = 'hierarchical/euclidean'
data_set = 'Irish_2010'
month = 1
n_clusters = 2

path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

attr = pd.read_csv(os.path.join(path, 'data', f'{data_set}_attr_final.csv'))
data = []
for i in trange(len(attr)):
    id = attr['ID'][i]
    df = pd.read_csv(os.path.join(path, 'data', f'{data_set}_monthly_interval', f'{id}.csv'), header = None).values
    data.append(df)
data = np.array(data)

100%|██████████| 918/918 [00:28<00:00, 32.29it/s]


In [5]:
path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'interval', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)

series = data[:, (month-1)*2:month*2, :months[month-1]*24]

In [4]:
for i in range(n_clusters):
    print('cluster:', i)
    index = list(clusters[month-1] == i)
    sub_series = series[index]
    sub_series = np.sum(sub_series, axis=0)
    test = sub_series[:, -168:]
    train = sub_series[:, :-168]

    scale = np.zeros(2)
    scale[0] = np.max(train)
    scale[1] = np.min(train)
    train = (train - scale[1])/(scale[0] - scale[1])
    test = (test - scale[1])/(scale[0] - scale[1])

    # test window (h = 1, 2, ..., 7)
    h = 1
    
    train_set = np.hstack((train, test[:, :(h-1)*24]))
    